In [1]:
import numpy
import pandas


In [2]:
df_reg = pandas.read_csv('RegularSeasonDetailedResults.csv')


In [4]:
loss_cols = ['Season', 'Lteam', 'Lscore', 'Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
             'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk',
             'Lpf']
win_cols = ['Season', 'Wteam', 'Wscore', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3',
            'Wftm', 'Wfta', 'Wor', 'Wdr', 'Wast', 'Wto', 'Wstl', 'Wblk',
            'Wpf']

fixed_cols = [col[1:] if col.startswith("W") else col for col in win_cols]

stat_cols = fixed_cols[2:]


In [5]:


df_win = df_reg[win_cols].copy()
df_win.columns = fixed_cols

df_loss = df_reg[loss_cols].copy()
df_loss.columns = fixed_cols


df_all = pandas.concat([df_win, df_loss])

In [6]:
df_mean = df_all.groupby(['team', "Season"], as_index=False).agg("mean")

In [7]:
df_mean.head()

,team,Season,score,fgm,fga,fgm3,fga3,ftm,fta,or,dr,ast,to,stl,blk,pf
0,1101,2014,63.142857,20.333333,50.142857,6.666667,17.857143,15.809524,21.190476,8.000000,20.333333,10.000000,15.000000,5.761905,1.476190,21.571429
1,1101,2015,61.000000,21.428571,52.928571,7.250000,19.178571,10.892857,14.964286,8.250000,19.642857,11.857143,12.821429,5.928571,1.178571,20.607143
2,1101,2016,69.851852,23.814815,53.925926,6.851852,18.851852,15.370370,21.740741,8.185185,22.518519,12.888889,13.407407,6.740741,2.444444,22.370370
3,1101,2017,67.880000,24.440000,53.320000,7.080000,19.080000,11.920000,18.560000,7.560000,22.880000,13.600000,14.480000,7.000000,2.760000,22.160000
4,1102,2003,57.250000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000


In [8]:
df_seeds = pandas.read_csv("TourneySeeds.csv")


# Seed is a string which includes division, e.g. "W08"
df_seeds["Seed"] = df_seeds["Seed"].str.slice(1, 3).astype(numpy.int32)

# Convert to seed and team IDs to int
df_seeds["Season"] = df_seeds["Season"].astype(numpy.int32)
df_seeds["Team"] = df_seeds["Team"].astype(numpy.int32)



In [9]:
# Add the seeds to the feature vectors 
df_features = df_mean.merge(df_seeds,
                            left_on=["team", "Season"],
                            right_on=["Team", "Season"], how="left")

# Set seeds for unseeded teams to 17
df_features.loc[df_features["Seed"].isnull(), "Seed"] = 17

# We don't need the team name string in the feature vectors
del df_features["Team"]


df_features.head()

,team,Season,score,fgm,fga,fgm3,fga3,ftm,fta,or,dr,ast,to,stl,blk,pf,Seed
0,1101,2014,63.142857,20.333333,50.142857,6.666667,17.857143,15.809524,21.190476,8.000000,20.333333,10.000000,15.000000,5.761905,1.476190,21.571429,17.0
1,1101,2015,61.000000,21.428571,52.928571,7.250000,19.178571,10.892857,14.964286,8.250000,19.642857,11.857143,12.821429,5.928571,1.178571,20.607143,17.0
2,1101,2016,69.851852,23.814815,53.925926,6.851852,18.851852,15.370370,21.740741,8.185185,22.518519,12.888889,13.407407,6.740741,2.444444,22.370370,17.0
3,1101,2017,67.880000,24.440000,53.320000,7.080000,19.080000,11.920000,18.560000,7.560000,22.880000,13.600000,14.480000,7.000000,2.760000,22.160000,17.0
4,1102,2003,57.250000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,11.428571,5.964286,1.785714,18.750000,17.0


In [10]:
df_features.to_csv("TeamSeasonFeatures.csv", index=False)